# Image Processing Module 7: Labeling and Quantifying Shape Feature Information

### Purpose: To move from thresholding to segmentation and shape feature quantification

Created by: Hawley Helmbrecht

Creation Date: 11/16/2020

Last Update: 

*Step 1: Import Necessary Packages*

In [37]:
import numpy as np
import pandas as pd
from scipy import ndimage
import os

import skimage.filters
from skimage import morphology
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage import io

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

*Step 2: Define User Inputs*

For this workflow you need to get the csv's of feature images folder by folder. So for each folder in the zipped file sent to you, copy in your correct folder location as a string into the folder location variable.  Remember, this provided formatting is for a Mac computer. Alter your folder location accordingly to how we learned over the summer based on your operating system

Then adjust the csv_name variable to the name would like this csv named as. I have used the name of the treatment group

You will not need to adjust the file_type_init and file_type_new variables

In [38]:
#replace the example path from my computer with the path to the image on your computer

folder_location = ''

csv_name = 'xyz.csv'

file_type_init = '.tif'
file_type_new = '.png'

Note: The cell images being read in are from the OGD Severity study by Rick Liao, Andrea Joseph, Mengying Zhang, Mike McKenna, Jeremy Filteau, and Hawley Helmbrecht within the Nance lab. It is an image taken from the thalamus with a costain DAPI/PI/Iba

*Step 3: The Code*

In [39]:
#Defines a function that cleans the folder input of any files excpet the ones of the type we want

def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

In [40]:
#Obtains the list of image names that the processing needs to be performed on

arr = os.listdir(folder_location)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type_init)

*Step 4: Getting Shape Features for our Images*

Here you will need to add your specific shape features into the Properties section in the props variable. Make sure the names are exactly as shown in the documentation for region props and as a string separated by commas

In [41]:
properties_list = ('area', 'bbox', 'bbox_area', 'centroid', 'convex_area', 'eccentricity', 'equivalent_diameter', 'euler_number', 'extent', 'filled_area', 'label', 'major_axis_length', 'minor_axis_length', 'moments', 'moments_central', 'moments_hu', 'moments_normalized', 'orientation', 'perimeter', 'solidity')



In [35]:
j = 0
for names in file_list:
    cell_im_location = str(folder_location + '/' + file_list[0])
    cell_im = io.imread(cell_im_location)
    blue_cell_im = cell_im[0,:, :]
    green_cell_im = cell_im[1,:,:]
    thresh_otsu = skimage.filters.threshold_otsu(green_cell_im)
    binary_otsu = green_cell_im > thresh_otsu
    new_binary_otsu = morphology.remove_small_objects(binary_otsu, min_size=64)
    label_image = label(new_binary_otsu)
    new_binary_otsu = ndimage.binary_fill_holes(new_binary_otsu)
    label_image = label(new_binary_otsu)
    image_label_overlay = label2rgb(label_image, image=new_binary_otsu, bg_label=0)
    from skimage import measure 

    #Insert your features here where the current red features are. You may want more features than what I told you to explore. 
    #Feel free to add them here as well. The computational time is pretty efficient
    props = measure.regionprops_table(label_image, properties=(properties_list))
   
    if j == 0:
        df = pd.DataFrame(props)
    else:
        df2 = pd.DataFrame(props)
        df = df.append(df2)
        
    j = 1

In [33]:
df

NameError: name 'df' is not defined

*Step 17: Saving as a CSV file*

The following code saves the pandas dataframe as a CSV that you can open with other software.  It will be saved in whatever directory you opened jupyter notebook or jupyuter lab from

In [8]:
df.to_csv(csv_name)

NameError: name 'df' is not defined

Now, I want you to repeat this process for each treatment group folder. Then it is up to you to begin visualizing the features with respect to the treatment groups

I recommend doing visualization in the following process:
1. By Generalized treatment group. (NT vs Injured vs Injured with Treatment)
2. By specific treatment groups (NT, OGD 0.5h, OGD 1.5h, OGD 3h ... etc)
3. Group by generalized treatment group and region (You will need to reorganize folders for this or add some lines to the code to provide you region based on file name)
4. Generally group regions without treatment group (hippocampus,  cortex, thalamus)
5. Specific treatment groups by region

And then in whatever way you were lead to based on your data